In [1]:
import pandas as pd

pd.show_versions()

C:\Users\soldesk\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.10.9.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.14393
machine          : AMD64
processor        : Intel64 Family 6 Model 58 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : Korean_Korea.949

pandas           : 1.5.3
numpy            : 1.23.5
pytz             : 2022.7
dateutil         : 2.8.2
setuptools       : 65.6.3
pip              : 22.3.1
Cython           : None
pytest           : 7.1.2
hypothesis       : None
sphinx           : 5.0.2
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.9.1
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.1.2
IPython          : 8.10.0
pandas_datareader: None
bs4              : 4.11.1
bottleneck       : 1.3.5
brotli       

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
nsmc_train_df = pd.read_csv('data/ratings_train.txt', encoding='UTF-8', sep='\t', engine='python')

nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
# null 값을 제거한다.
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

In [6]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [7]:
nsmc_train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

- 한글 외의 문자 제거

In [8]:
import re

In [9]:
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]',' ', x))

nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


- 특성 벡터화 작업: 형태소 분석

In [10]:
from konlpy.tag import Okt

# 형태소 분석에 사용할 클래스 객체 생성
okt = Okt()

In [11]:
# 각 형태소 토큰화
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [12]:
# TF - IDF 기반 벡터화를 위한 사이킷런 객체 생성
from sklearn.feature_extraction.text import TfidfVectorizer

#tokenizer : 토큰 생성기
# ngram_range: 토큰의 단어 크기
# min_df: 토큰의 최소 출현 빈도
# max_df: 최대 빈도(퍼센트)
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

- 감성 분류 모델 구축하기

In [13]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [14]:
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(random_state=0)

In [16]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [17]:
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [18]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 3} 0.8553


In [19]:
# 최적 매개변수의 best 모델 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [20]:
# 평가용 데이터의 피처 벡터화
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

In [22]:
test_predict = SA_lr_best.predict(nsmc_train_tfidf)

In [24]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도:', round(accuracy_score(nsmc_train_df['label'], test_predict), 3))

감성 분석 정확도: 0.932


- 새로운 텍스트로 감정 예측 확인

In [35]:
st = input('감성 분석할 문장 입력 >>')

감성 분석할 문장 입력 >>안녕하세요? 반갑습니다. 좋은 하루 되세요^^


In [36]:
# 입력 텍스트에 대한 전처리
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st)
print(st)
st = [' '.join(st)]
print(st)

['안녕하세요', '반갑습니다', '좋은', '하루', '되세요']
['안녕하세요 반갑습니다 좋은 하루 되세요']


In [37]:
# 입력 텍스트의 피처 벡터화
st_tfidf = tfidf.transform(st)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = SA_lr_best.predict(st_tfidf)

In [38]:
# 예측값 출력
if(st_predict == 0):
    print(st, "->> 부정 감성")
else:
    print(st, "->> 긍정 감성")

['안녕하세요 반갑습니다 좋은 하루 되세요'] ->> 긍정 감성
